In [7]:
import os
import numpy as np
import time
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow.keras.optimizers  as opt

In [2]:
# create the base pre-trained model
base_model = ResNet50(weights=None, include_top=False, input_shape=(256,256,1))


In [3]:
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 2 classes
predictions = Dense(2, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
#for layer in base_model.layers:
#    layer.trainable = False


In [4]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 1) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 1)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 128, 128, 64) 3200        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 128, 128, 64) 256         conv1_conv[0][0]                 
_______________________________________________________________________________________

In [2]:
batch_size = 8

train_datagen = image.ImageDataGenerator(
        #rotation_range=10,
        #width_shift_range=0.2,
        #height_shift_range=0.2,
        rescale=1./255,
        #shear_range=0.2,
        #zoom_range=0.2,
        #preprocessing_function=preprocess_input,
        horizontal_flip=True,
        fill_mode='constant')

train_generator = train_datagen.flow_from_directory(
        'FilteredTrainDataNew',  # this is the target directory
        target_size=(256, 256),  # all images will be resized to 150x150
        color_mode ='grayscale',
        batch_size=batch_size,
        class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels

Found 4104 images belonging to 2 classes.


In [3]:
preview_images = 0
if preview_images:
    img = image.load_img('FilteredTrainData/00000/FLAIR/Image-228.png')  # this is a PIL image
    x = image.img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
    x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)
    i = 0
    for batch in train_datagen.flow(x, batch_size=1,save_to_dir='preview', save_prefix='cat', save_format='jpeg'):
        i += 1
        if i > 20:
            break  # otherwise the generator would loop indefinitely
            

In [4]:
x,y = train_generator[0]
train_generator.class_indices

{'NoTumor': 0, 'Tumor': 1}

In [5]:
y

array([[1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.]], dtype=float32)

In [8]:
path = './trained_models/best_model.h5'
model = load_model(path)

In [9]:
outmodel = 'trained_models'
os.makedirs(outmodel,exist_ok=True)

def get_lr_metric(optimizer):
    def lr(y_true, y_pred):
        return optimizer.lr
    return lr


initial_learning_rate = 0.01
lr_schedule = opt.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=1000,
    decay_rate=0.96,
    staircase=True)

#sgd_opt = opt.SGD(learning_rate=lr_schedule)
#adam_opt = opt.Adam(learning_rate=lr_schedule)
#lr_metric = get_lr_metric(sgd_opt)
model.compile(loss= 'categorical_crossentropy',#'binary_crossentropy',
              optimizer='adam',#sgd_opt,
              metrics=['accuracy'])

checkpoint = ModelCheckpoint(f"{outmodel}/best_model.h5", monitor='accuracy', verbose=0, save_best_only=True, mode='max', save_freq=2)

model.fit(
        train_generator,
        steps_per_epoch= len(train_generator),
        epochs=10,callbacks=[checkpoint])

model.save_weights(f'{outmodel}/FINAL_MODEL.h5')  # always save your weights after training or during training

Epoch 1/10
513/513 [==============================] - 148s 288ms/step - loss: 0.3841 - accuracy: 0.8477
Epoch 2/10
513/513 [==============================] - 156s 304ms/step - loss: 0.3310 - accuracy: 0.8801
Epoch 3/10
513/513 [==============================] - 157s 306ms/step - loss: 0.3175 - accuracy: 0.8796
Epoch 4/10
513/513 [==============================] - 156s 304ms/step - loss: 0.2887 - accuracy: 0.8952
Epoch 5/10
513/513 [==============================] - 158s 307ms/step - loss: 0.2602 - accuracy: 0.9047
Epoch 6/10
513/513 [==============================] - 162s 317ms/step - loss: 0.2392 - accuracy: 0.9115
Epoch 7/10
513/513 [==============================] - 170s 331ms/step - loss: 0.2007 - accuracy: 0.9284
Epoch 8/10
513/513 [==============================] - 177s 346ms/step - loss: 0.1982 - accuracy: 0.9301
Epoch 9/10
513/513 [==============================] - 184s 359ms/step - loss: 0.1870 - accuracy: 0.9359
Epoch 10/10
513/513 [==============================] - 193s 377m

In [10]:
model.save(f'{outmodel}/FINAL_MODELALL.h5')  # always save your weights after training or during training